In [297]:
from sklearn.ensemble.forest import _generate_unsampled_indices
from sklearn.ensemble.forest import _generate_sample_indices
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

from tqdm import tqdm

from itertools import permutations

import numpy as np
np.warnings.filterwarnings('ignore')

This notebook investigates transfer learning (and, eventually, lifelong learning) using three methods: Ignoring possible useful information in disparate data sets, using the structure in learned trees to "transfer" information, and comparing two classifiers to assess how similar the underlying distributions are.

In the two latter methods, the ideas underlying Guo's Conditional Entropy Forests are used to estimate posteriors.

To see a full description of the task, see https://github.com/neurodata/lifelong-learning/blob/master/willy_nilly.ipynb

In [334]:
"""
This file contains code for conditional entropy forests
"""

def finite_sample_correction(class_probs, row_sums):
    
    where_0 = np.argwhere(class_probs == 0)
    for elem in where_0:
        class_probs[elem[0], elem[1]] = 1 / (2 * row_sums[elem[0], None])
    where_1 = np.argwhere(class_probs == 1)
    for elem in where_1:
        class_probs[elem[0], elem[1]] = 1 - 1 / (2 * row_sums[elem[0], None])
    
    return class_probs

def build_model(X, y, n_estimators=200, max_samples=.32,
                                            bootstrap=True,
                                            depth=30,
                                            min_samples_leaf=1):
    if X.ndim == 1:
        X=X.reshape(-1, 1)
        
    model=BaggingClassifier(DecisionTreeClassifier(max_depth=depth, min_samples_leaf=min_samples_leaf,
                                                     max_features = np.ceil(int(np.sqrt(X.shape[1])))),
                              n_estimators=n_estimators,
                              max_samples=max_samples,
                              bootstrap=bootstrap)
    model.fit(X, y)
    return model

def estimate_posteriors(model, train, y, test, in_task=True, acorn=None):
    if acorn is None:
        acorn = np.random.randint(10**6)
    np.random.seed(acorn)
    
    n, d = train.shape
    m, d_ = test.shape
    
    if d != d_:
        raise ValueError("train and test data in different dimensions")
    
    class_counts = np.zeros((m, model.n_classes_))
    for tree in model:
        # get out of bag indicies
        if in_task:
            prob_indices = _generate_unsampled_indices(tree.random_state, n)
            # in_bag_idx = _generate_sample_indices(tree.random_state, n) # this is not behaving as i expected
        else:
            prob_indices = range(n)
            in_bag_idx = range(n)
        
        all_leaf_nodes = tree.apply(train)
        unique_leaf_nodes = np.unique(all_leaf_nodes)
            
        # get all node counts
        node_counts = tree.tree_.n_node_samples
        # get probs for eval samples
        posterior_class_counts = np.zeros((len(unique_leaf_nodes), model.n_classes_))

        for prob_index in prob_indices:
            temp_node = tree.apply(train[prob_index].reshape(1, -1)).item()
            posterior_class_counts[np.where(unique_leaf_nodes == temp_node)[0][0], y[prob_index]] += 1
            
        # total number of points in a node
        row_sums = posterior_class_counts.sum(axis=1)
        
        # no divide by zero
        row_sums[row_sums == 0] = 1

        # posteriors
        class_probs = (posterior_class_counts / row_sums[:, None])
        # posteriors with finite sampling correction
        
        class_probs = finite_sample_correction(class_probs, row_sums)
    
        # posteriors as a list
        class_probs.tolist()
        partition_counts = np.asarray([node_counts[np.where(unique_leaf_nodes == x)[0][0]] for x in tree.apply(test)])
        # get probability for out of bag samples
        eval_class_probs = [class_probs[np.where(unique_leaf_nodes == x)[0][0]] for x in tree.apply(test)]
        eval_class_probs = np.array(eval_class_probs)
        # find total elements for out of bag samples
        elems = np.multiply(eval_class_probs, partition_counts[:, np.newaxis])
        # store counts for each x (repeat fhis for each tree)
        class_counts += elems
    # calculate p(y|X = x) for all x's
    probs = class_counts / class_counts.sum(axis=1, keepdims=True)
    
    return probs

def predict(posteriors):
    return np.argmax(s, axis = 1)

def permutation(predict1, predict2):
    """
    how to use:
    
    this function returns the permutation i.e. \pi: [K] -> [K] that maximizes
    the number of matched predictions
    
    to use the permutation for posteriors for point i (posterior_i), say, simply use
    posterior_i[permutation]
    
    """
    unique_1 = np.unique(predict1)
    unique_2 = np.unique(predict2)
    
    if set(unique_1) != set(unique_2):
        raise ValueError("predictions must be on the same set of labels")
        
    K = len(unique_1)
    
    max_sum = 0
    max_perm = unique_2
    for i, perm in enumerate(permutations(unique_2)):
        perm = np.array(list(perm))
        temp_predict2 = -1*np.ones(len(predict2))
        
        for k in range(K):
            temp_predict2[np.where(predict2 == unique_2[k])[0]] = perm[k]
           
        temp_sum = np.sum(predict1 == temp_predict2)
        if temp_sum > max_sum:
            max_sum = temp_sum
            max_perm = perm
            
    return max_perm
            
def estimate_alpha(predict1, predict2, permutation=None):
    if permutation is None:
        return np.sum(predict1 == predict2) / len(predict1)
    else:
        unique = np.unique(temp_predict2)
        temp_predict2 = -1*np.ones(len(predict2))
        
        for i, k in enumerate(unique):
            temp_predict2[predict2 == k] = permutation[i]
            
        return np.sum(predict1 == temp_predict2) / len(predict1)
    
def generate_sample(n, pi, conditional_0, params0, conditional_1, params1, d = 2, acorn=None):
    if acorn is None:
        acorn = np.random.seed(10**6)
    np.random.seed(acorn)
    
    n0 = int(np.random.binomial(n, pi))
    n1 = n - n0
    ns = [n0, n1]
    
    X0 = conditional_0(*params0, size=(n0, d))
    
    X1 = conditional_1(*params1, size=(n1, d))
    
    labels = np.concatenate([i*np.ones(ns[i]) for i in range(len(ns))]).astype(int)
    
    return [np.concatenate((X0, X1), axis = 0), labels]

In [336]:
np.random.seed(1)

mc_its = 100 # number of simulation repititions
ns0 = 10*np.arange(1,6,step=1).astype(int)
ns1 = 100*np.arange(0.5, 5.5, step=0.5).astype(int) # number of training samples 
ns = np.concatenate((ns0, ns1))
n = 100
nz_prop = 1

m = 1000 # number of test samples each monte carlo iteration

pi = 0.5
cond_X_0, cond_X_1 = 2*[np.random.uniform]
params_X_0, params_X_1 = [0, 0.5], [0.5, 1]

cond_Z_0, cond_Z_1 = 2*[np.random.uniform]
params_Z_0, params_Z_1 = [0.5, 1], [0, 0.5]

algorithms = ['silly willy nilly t1', 'silly willy nilly t2', 'task1', 'task2', 'task1 cep global', 'task2 cep global', 'task1 jtv', 'task2 jtv']
algorithms = np.concatenate((algorithms, ['task1 cep local', 'task2 cep local', 'task1 jtv', 'task2 jtv']))

errors = [[[] for _ in ns] for __ in algorithms]
mean_error = np.zeros((len(algorithms), len(ns)))
std_error = np.zeros((len(algorithms), len(ns)))

for j, n in enumerate(tqdm(ns)):
    errors = [[[] for _ in ns] for __ in algorithms]
    
    k = int(np.floor(np.log(n)))
    T = int(np.floor(np.sqrt(n)))
    
    for i in range(mc_its):
        temp_predictions = [[] for __ in algorithms]
        
        X, labelsX = generate_sample(n, pi, cond_X_0, params_X_0, cond_X_1, params_X_1)
        testX, test_labelsX = generate_sample(m, pi, cond_X_0, params_X_0, cond_X_1, params_X_1)
        
        Z, labelsZ = generate_sample(n, pi, cond_Z_0, params_Z_0, cond_Z_1, params_Z_1)
        testZ, test_labelsZ = generate_sample(m, pi, cond_Z_0, params_Z_0, cond_Z_1, params_Z_1)
        
        joint, joint_labels = np.concatenate((X, Z)), np.concatenate((labelsX, labelsZ))
        joint_test = np.concatenate((testX, testZ), axis = 0)
        
        model_joint = build_model(joint, joint_labels)
        model_X = build_model(X, labelsX)
        model_Z = build_model(Z, labelsZ)
        
        posteriors_structjoint_estjoint=estimate_posteriors(model_joint, joint, joint_labels, joint_test, in_task=True)
        predictions_joint = predict(posteriors_structjoint_estjoint)
        
        # calculate errors for jointly learned forests
        errors[0][i].append(np.sum(test_labelsX == predictions_joint[range(n)])/n)
        errors[1][i].append(np.sum(test_labelsZ == predictions_joint[range(n, 2*n)])/n)
        
        posteriors_structX_estX=estimate_posteriors(model_X, X, labelsX, testX, in_task=True)
        posteriors_structZ_estX=estimate_posteriors(model_Z, X, labelsX, testX, in_task=False)
        
        pred_structX_estX=predict(posteriors_structX_estX)
        pred_structZ_estX=predict(posteriors_structZ_estX)
        
        posteriors_structX_estZ=estimate_posteriors(model_X, Z, labelsZ, testZ, in_task=False)
        posteriors_structZ_estZ=estimate_posteriors(model_Z, Z, labelsZ, testZ, in_task=True)
        
        pred_structX_estZ=predict(posteriors_structX_estZ)
        pred_structZ_estZ=predict(posteriors_structZ_estZ)
        
        # calculate errors without attempting to transfer knowledge
        pred_X = predict(posteriors_structX_estX)
        pred_Z = predict(posteriors_structZ_estZ)
        
        errors[2][i].append(np.sum(test_labelsX == pred_X)/n)
        errors[3][i].append(np.sum(test_labelsZ == pred_Z)/n)
        
        # jtv ?
        
        # cep global
        optimal_permutation_X = permutation(pred_structX_estX, pred_structZ_estX)
        optimal_permutation_Z = permutation(pred_structZ_estZ, pred_structX_estZ)
        
        new_posteriors_structX_estZ = np.zeros(posteriors_structX_estZ.shape)
        for i in range(n):
            new_posteriors_structX_estZ[i] = posteriors_structX_estZ[i][optimal_permutation_X]
            new_posteriors_structZ_estX[i] = posteriors_structZ_estX[i][optimal_permutation_Z]
            
        pred_structX_estZ = predict(posteriors_structX_estZ)
        pred_structZ_estX = predict(posteriors_structZ_estX)
        
        alpha_X = estimate_alpha(pred_structX_estX, pred_structZ_estX)
        alpha_Z = estimate_alpha(pred_structZ_estZ, pred_structX_estZ)
            
        pred_X_cep_global = predict(posteriors_structX_estX + alpha_X * new_posteriors_structZ_estX)
        pred_Z_cep_global = predict(posteriors_structZ_estZ + alpha_Z * new_posteriors_structX_estZ)





  0%|          | 0/15 [00:00<?, ?it/s]

IndexError: index 0 is out of bounds for axis 0 with size 0

In [ ]:
model = build_model(X, labelsX)
s = estimate_posteriors(model, X, labelsX, testZ)

In [280]:
s = estimate_posteriors(model, Z, labelsZ, testZ, in_task=False)

(1000, 2)
